In [386]:
# Treasury Forecasting algo version #2 determining the best investnent windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=False)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [387]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [388]:
# STEP 1: Get the asset classes combined with their parent class ID
table_name = 'AssetClass'
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
asset_classes = fetch_data(table_name, '',1,sql)
asset_classes

,ID,Title,Group,Issuer,PercentMax,AssetClassCombined
0,1,Cash/Sweep,Not Assigned,None,1.0000,Cash/Sweep
1,2,Certificate of Deposit,Certificates of Deposit,None,0.5000,Certificate of Deposit
2,3,Commercial Paper,Commercial Paper,None,0.3330,Commercial Paper
3,4,US Agencies,US Agencies,None,0.2000,US Agencies
4,7,Money Market,Not Assigned,None,0.2000,Money Market
5,8,Mutual Fund,Not Assigned,None,0.2000,Mutual Fund
6,16,US Treasuries,US Treasuries,None,1.0000,US Treasuries


In [389]:
# STEP 4: Running balance Day view taken from the SQL views
#           Q: Do we want to replace the SQL views with panasda dataframes?

table_name = 'RunningBalanceDayView'
running_balances = fetch_data(table_name)


In [390]:
# Add the daily total portfolio balance to the running balances DataFrame

# Convert TransactionDate to datetime if not already
running_balances['TransactionDate'] = pd.to_datetime(running_balances['TransactionDate'])

# Filter and pivot Portfolio balances
portfolio_balances = running_balances[running_balances['TransactionClass'] == 'Portfolio'].copy()
portfolio_balances['RunningTotal'] = pd.to_numeric(portfolio_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_portfolio = portfolio_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_portfolio.reset_index().rename(columns={'RunningTotal': 'Portfolio'}),
    on='TransactionDate',
    how='left'
)

In [391]:
# Add the asset class PercentMax column to the running balances DataFrame matching the on TransactionClass colunmn

# Create a mapping dictionary from Title to PercentMax
percentmax_mapping = dict(zip(asset_classes['Title'], asset_classes['PercentMax']))

# Update PercentMax in running_balances using the mapping
running_balances['PercentMax'] = running_balances['TransactionClass'].map(percentmax_mapping).fillna(1.0)

# Convert PercentMax to float
# running_balances['PercentMax'] = pd.to_numeric(running_balances['PercentMax'])

# Check the PercentMax mapping
# percentmax_mapping
running_balances[running_balances['TransactionClass'] == 'Certificate of Deposit'].head(10)
# running_balances.info()

,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax
1244,2025-01-21,Certificate of Deposit,2600000.00,"654,628,445",0.5000
1245,2025-01-22,Certificate of Deposit,2600000.00,"492,520,445",0.5000
1246,2025-01-23,Certificate of Deposit,2600000.00,"492,520,445",0.5000
1247,2025-01-24,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1248,2025-01-25,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1249,2025-01-26,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1250,2025-01-27,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1251,2025-01-28,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1252,2025-01-29,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1253,2025-01-30,Certificate of Deposit,2600000.00,"464,146,445",0.5000


In [392]:
running_balances[running_balances['TransactionClass'] == 'Certificate of Deposit'].head(10)

,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax
1244,2025-01-21,Certificate of Deposit,2600000.00,"654,628,445",0.5000
1245,2025-01-22,Certificate of Deposit,2600000.00,"492,520,445",0.5000
1246,2025-01-23,Certificate of Deposit,2600000.00,"492,520,445",0.5000
1247,2025-01-24,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1248,2025-01-25,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1249,2025-01-26,Certificate of Deposit,2600000.00,"470,088,445",0.5000
1250,2025-01-27,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1251,2025-01-28,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1252,2025-01-29,Certificate of Deposit,2600000.00,"464,146,445",0.5000
1253,2025-01-30,Certificate of Deposit,2600000.00,"464,146,445",0.5000


In [393]:
# Compute the asset class's maximum based on the policy
# Convert PercentMax to float first, then calculate PolicyMax
running_balances['PercentMax'] = running_balances['PercentMax'].astype(float)

# ERROR: Float is needed, but the object->float conversion resets the value to 0.0!

running_balances['PolicyMax'] = running_balances['Portfolio'] * running_balances['PercentMax']

# running_balances.info()


In [394]:
running_balances[running_balances['TransactionClass'] == 'Certificate of Deposit'].head(10)

,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax,PolicyMax
1244,2025-01-21,Certificate of Deposit,2600000.00,"654,628,445",0,"327,314,223"
1245,2025-01-22,Certificate of Deposit,2600000.00,"492,520,445",0,"246,260,223"
1246,2025-01-23,Certificate of Deposit,2600000.00,"492,520,445",0,"246,260,223"
1247,2025-01-24,Certificate of Deposit,2600000.00,"470,088,445",0,"235,044,223"
1248,2025-01-25,Certificate of Deposit,2600000.00,"470,088,445",0,"235,044,223"
1249,2025-01-26,Certificate of Deposit,2600000.00,"470,088,445",0,"235,044,223"
1250,2025-01-27,Certificate of Deposit,2600000.00,"464,146,445",0,"232,073,223"
1251,2025-01-28,Certificate of Deposit,2600000.00,"464,146,445",0,"232,073,223"
1252,2025-01-29,Certificate of Deposit,2600000.00,"464,146,445",0,"232,073,223"
1253,2025-01-30,Certificate of Deposit,2600000.00,"464,146,445",0,"232,073,223"


In [395]:
running_balances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4976 entries, 0 to 4975
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   TransactionDate   4976 non-null   datetime64[ns]
 1   TransactionClass  4976 non-null   object        
 2   RunningTotal      4976 non-null   object        
 3   Portfolio         4976 non-null   float64       
 4   PercentMax        4976 non-null   float64       
 5   PolicyMax         4976 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 233.4+ KB


In [396]:
# Add the daily available cash balance to the running balances DataFrame

# Filter and pivot cash balances
cash_balances = running_balances[running_balances['TransactionClass'] == 'Cash/Sweep'].copy()
cash_balances['RunningTotal'] = pd.to_numeric(cash_balances['RunningTotal'])

# Create a series with daily portfolio balances
daily_cash = cash_balances.set_index('TransactionDate')['RunningTotal']

# Add portfolio balance to runningBalances DataFrame
running_balances = running_balances.merge(
    daily_cash.reset_index().rename(columns={'RunningTotal': 'CashSweep'}),
    on='TransactionDate',
    how='left'
)

In [397]:
# Add the amount investable column
#       if [TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then [PolicyMax]-[RunningTotal] else [RunningTotal]

# Convert RunningTotal to numeric if it's not already
running_balances['RunningTotal'] = pd.to_numeric(running_balances['RunningTotal'])

# Add Investable column based on the condition
running_balances['Investable'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') &
    (running_balances['TransactionClass'] != 'Cash/Sweep'),
    running_balances['PolicyMax'] - running_balances['RunningTotal'],
    running_balances['RunningTotal']
)


In [398]:
# Add the final available column
#       if ([TransactionClass] <> "Portfolio" && [TransactionClass] <> "Cash/Sweep",
#       then MIN([CashSweep],[Investable]) else [RunningTotal])
running_balances['Available'] = np.where(
    (running_balances['TransactionClass'] != 'Portfolio') &
    (running_balances['TransactionClass'] != 'Cash/Sweep'),
    np.minimum(running_balances['CashSweep'], running_balances['Investable']),
    running_balances['RunningTotal']
)

In [405]:
# Output the final datafram

# Suppress scientific notation by setting float_format
pd.options.display.float_format = '{:,.0f}'.format

# Display the dataframe without scientific notation
print(running_balances[running_balances['TransactionClass'] == 'Commercial Paper'].head(10))

     TransactionDate  TransactionClass  RunningTotal   Portfolio  PercentMax  \
1866      2025-01-21  Commercial Paper   170,669,204 654,628,445           0   
1867      2025-01-22  Commercial Paper   161,169,204 492,520,445           0   
1868      2025-01-23  Commercial Paper   151,169,204 492,520,445           0   
1869      2025-01-24  Commercial Paper   142,669,204 470,088,445           0   
1870      2025-01-25  Commercial Paper   142,669,204 470,088,445           0   
1871      2025-01-26  Commercial Paper   142,669,204 470,088,445           0   
1872      2025-01-27  Commercial Paper   142,669,204 464,146,445           0   
1873      2025-01-28  Commercial Paper   140,869,204 464,146,445           0   
1874      2025-01-29  Commercial Paper   140,869,204 464,146,445           0   
1875      2025-01-30  Commercial Paper   140,869,204 464,146,445           0   

       PolicyMax   CashSweep  Investable  Available  
1866 217,991,272 151,070,375  47,322,068 47,322,068  
1867 164,00

In [401]:
running_balances

,TransactionDate,TransactionClass,RunningTotal,Portfolio,PercentMax,PolicyMax,CashSweep,Investable,Available
0,2025-01-21,Portfolio,"654,628,445","654,628,445",1,"654,628,445","151,070,375","654,628,445","654,628,445"
1,2025-01-22,Portfolio,"492,520,445","492,520,445",1,"492,520,445","32,442,375","492,520,445","492,520,445"
2,2025-01-23,Portfolio,"492,520,445","492,520,445",1,"492,520,445","42,442,375","492,520,445","492,520,445"
3,2025-01-24,Portfolio,"470,088,445","470,088,445",1,"470,088,445","28,510,375","470,088,445","470,088,445"
4,2025-01-25,Portfolio,"470,088,445","470,088,445",1,"470,088,445","28,510,375","470,088,445","470,088,445"
...,...,...,...,...,...,...,...,...,...
4971,2026-09-30,US Treasuries,"-1,883,267","412,783,445",1,"412,783,445","372,132,019","414,666,712","372,132,019"
4972,2026-10-01,US Treasuries,"-1,883,267","412,783,445",1,"412,783,445","372,132,019","414,666,712","372,132,019"
4973,2026-10-02,US Treasuries,"-1,883,267","412,783,445",1,"412,783,445","372,132,019","414,666,712","372,132,019"
4974,2026-10-03,US Treasuries,"-1,883,267","412,783,445",1,"412,783,445","372,132,019","414,666,712","372,132,019"
